In [1]:
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Embedding,GRU,TimeDistributed,RepeatVector, LSTM, concatenate , Input, Reshape
from keras.preprocessing.text import one_hot
from keras.preprocessing import sequence
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
from keras import backend as K 
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

Using TensorFlow backend.
/Users/ewallner/.brew/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
max_caption_len = 3
vocab_size = 3

In [3]:
images = []
for i in range(2):
    images.append(img_to_array(load_img('screenshot.jpg', target_size=(224, 224))))
images = np.array(images, dtype=float)
images = preprocess_input(images)

In [4]:
html_input = np.array(
            [[[0., 0., 0.], #start
             [0., 0., 0.],
             [1., 0., 0.]],
             [[0., 0., 0.], #start <HTML>Hello World!</HTML>
             [1., 0., 0.],
             [0., 1., 0.]]])

next_words = np.array(
            [[0., 1., 0.], # <HTML>Hello World!</HTML>
             [0., 0., 1.]]) # end

In [31]:
VGG = VGG16(weights='imagenet', include_top=True)
vgg_out = RepeatVector(max_caption_len)(vgg_out)

553467904/553467096 [==============================] - 59s 0us/step


ValueError: Input 0 is incompatible with layer repeat_vector_4: expected ndim=2, found ndim=3

In [ ]:
language_input = Input(shape=(vocab_size, vocab_size))
language_model = LSTM(5, return_sequences=True)(language_input)

In [ ]:
decoder = concatenate([vgg_out, language_model])
decoder = LSTM(5, return_sequences=False)(decoder)
decoder_output = Dense(vocab_size, activation='softmax')(decoder)
model = Model(inputs=[VGG.input, language_input], outputs=decoder_output)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
model.fit([images, html_input], next_words, batch_size=1, shuffle=False, epochs=20)

In [ ]:
first_word = model.predict([np.array([images[1]]), np.array([html_input[1]])])

In [30]:
print(first_word)

[[ 0.13694935  0.64393753  0.21911311]]


In [26]:
from numpy import argmax

In [ ]:
this = argmax(first_word)

In [ ]:
print(this)